# Playground for strokes graph

In [18]:
# import libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import folium
from itertools import combinations
from shapely import LineString

Import example data set

In [19]:
# read in toy graph (Bubenec)
streets = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")

# ax = streets.plot(figsize=(8, 8), column="id", cmap = "Accent")
# ax.set_axis_off()
# ax.legend()

Workflow (will be wrapped in function later)

- [ ] input is a set of linestrings
- [ ] remove false nodes 
- [ ] convert linestrings to primal graph
- [ ] get points and lines gdf from primal graph
- [ ] run COINS on lines gdf to find strokes
- [ ] add mapping of strokeID:edgeIDs to stroke gdf (use momepy's **edge ID**, not indexing)
- [ ] add stroke attribute to each edge on primal graph

In [20]:
# define variable defaults (will be arguments passed to future function)
angle_threshold=0
flow_mode=False

In [21]:
# remove false nodes
streets = momepy.remove_false_nodes(streets)

# add unique edge ID to streets, already HERE!
streets["edge_id"] = streets.index

# make primal graph
graph = momepy.gdf_to_nx(streets, approach="primal")

# get gdfs of points and lines
points, lines = momepy.nx_to_gdf(graph, points=True, lines=True)

# make coins
coins = momepy.COINS(lines, angle_threshold=angle_threshold, flow_mode=flow_mode)

# get gdfs from COINS class
stroke_attribute = coins.stroke_attribute()
stroke_gdf = coins.stroke_gdf()
stroke_gdf["rep_point"] = stroke_gdf.geometry.apply(lambda x: x.interpolate(0.5, normalized=True))

# add stroke_id column
stroke_gdf["stroke_id"] = stroke_gdf.index

# add edge_ids column (using COINS.stroke_attribute to map into ID defined in lines gdf)
stroke_gdf["edge_ids"] = stroke_gdf.stroke_id.apply(
    lambda x: list(
        lines.iloc[
            stroke_attribute[stroke_attribute == x].index]["edge_id"]
    )
)

stroke_gdf.head()

,n_segments,geometry,rep_point,stroke_id,edge_ids
stroke_group,,,,,
0,8,"LINESTRING (1603278.899 6463669.186, 1603283.7...",POINT (1603374.663 6464077.898),0,"[0, 3, 15, 27]"
1,17,"LINESTRING (1603537.194 6464558.112, 1603557.6...",POINT (1603707.107 6464238.854),1,"[11, 28, 2, 30]"
2,5,"LINESTRING (1603413.206 6464228.73, 1603274.45...",POINT (1603149.929 6464130.225),2,"[4, 5, 6]"
3,5,"LINESTRING (1603287.304 6464587.705, 1603286.8...",POINT (1603342.343 6464406.368),3,[26]
4,19,"LINESTRING (1603077.5 6464475.323, 1603085.515...",POINT (1603237.049 6464133.622),4,"[1, 12, 14, 25]"


In [22]:
# make dictionary for primal graph, edge_id:edge_name
d = nx.get_edge_attributes(graph, "edge_id")
d = {v:k for k,v in d.items()}
d

{0: ((1603585.6402153103, 6464428.773867372),
  (1603413.2063240695, 6464228.730248732),
  0),
 11: ((1603585.6402153103, 6464428.773867372),
  (1603650.450422848, 6464368.600601688),
  0),
 28: ((1603585.6402153103, 6464428.773867372),
  (1603537.1939729159, 6464558.11228298),
  0),
 2: ((1603413.2063240695, 6464228.730248732),
  (1603607.3029882177, 6464181.852772597),
  0),
 3: ((1603413.2063240695, 6464228.730248732),
  (1603363.557831175, 6464031.88480676),
  0),
 4: ((1603413.2063240695, 6464228.730248732),
  (1603226.9576840235, 6464160.158361825),
  0),
 26: ((1603413.2063240695, 6464228.730248732),
  (1603287.303979983, 6464587.704889874),
  0),
 1: ((1603268.502117987, 6464060.781328565),
  (1603363.557831175, 6464031.88480676),
  0),
 12: ((1603268.502117987, 6464060.781328565),
  (1603226.9576840235, 6464160.158361825),
  0),
 20: ((1603268.502117987, 6464060.781328565),
  (1603146.6963311615, 6463924.630126579),
  0),
 14: ((1603363.557831175, 6464031.88480676),
  (1603558

In [23]:
# for each edge, add "stroke_id" as attribute to graph
for _, row in stroke_gdf.iterrows():
    for e in row.edge_ids: 
        graph.edges[d[e]]["stroke_id"] = row.stroke_id

In [24]:
d_strokes = nx.get_edge_attributes(graph, "stroke_id")
d_edges = nx.get_edge_attributes(graph, "edge_id")

In [25]:
edges_strokes = {d_edges[k]:d_strokes[k] for k in d_edges} 

In [26]:
m = stroke_gdf.explore(tiles="cartodb.positron", column = "stroke_id", name = "strokes", cmap = "Reds", style_kwds={"weight":8})
lines.explore(m=m, column = "edge_id", name = "lines", cmap = "Blues", style_kwds={"weight":8})
folium.LayerControl().add_to(m)
m

* Now we have a primal graph `graph` where each edge has the attributes `edge_id` and `stroke_id`
* We have this information also in `stroke_gdf`
* Each stroke (each line in stroke_gdf) will be a node of the stroke graph

In [27]:
stroke_graph = nx.Graph()
stroke_graph.graph["crs"] = graph.graph["crs"]
stroke_graph.graph["approach"] = graph.graph["approach"]
stroke_graph.add_nodes_from(
    [
        (
            row.stroke_id, 
            {
                "edge_ids": row.edge_ids,
                "geometry": row.rep_point,
                "x": row.rep_point.xy[0][0],
                "y": row.rep_point.xy[1][0],
            }
        ) for _, row in stroke_gdf.iterrows()
    ]
)
# node names are the stroke IDs.
# each node has the attribute "edge_ids".
stroke_graph.nodes(data=True)

NodeDataView({0: {'edge_ids': [0, 3, 15, 27], 'geometry': <POINT (1603374.663 6464077.898)>, 'x': 1603374.6625343116, 'y': 6464077.898491419}, 1: {'edge_ids': [11, 28, 2, 30], 'geometry': <POINT (1603707.107 6464238.854)>, 'x': 1603707.1065106073, 'y': 6464238.853991265}, 2: {'edge_ids': [4, 5, 6], 'geometry': <POINT (1603149.929 6464130.225)>, 'x': 1603149.9288811635, 'y': 6464130.224503239}, 3: {'edge_ids': [26], 'geometry': <POINT (1603342.343 6464406.368)>, 'x': 1603342.3426854417, 'y': 6464406.368225728}, 4: {'edge_ids': [1, 12, 14, 25], 'geometry': <POINT (1603237.049 6464133.622)>, 'x': 1603237.0487682838, 'y': 6464133.622486805}, 5: {'edge_ids': [20], 'geometry': <POINT (1603207.597 6463992.708)>, 'x': 1603207.5969886228, 'y': 6463992.707728057}, 6: {'edge_ids': [16, 17, 29, 18, 23], 'geometry': <POINT (1603592.235 6464121.336)>, 'x': 1603592.2349246691, 'y': 6464121.336160048}, 7: {'edge_ids': [7, 8, 21, 9, 24, 22, 13], 'geometry': <POINT (1603264.658 6463848.976)>, 'x': 16032

to find the edges of the stroke graph, we look at the primal graph's nodes and the stroke_ids of their adjacent edges

In [28]:
for n in graph.nodes:
    print(n)
    es = list(graph.edges(n, keys=True))
    stroke_list = [graph.edges[e]["stroke_id"] for e in es]
    stroke_set = set(stroke_list)
    print(stroke_list, stroke_set)
    # for all size2 combinations from stroke_set
    for c in combinations(stroke_set, 2):
        print(c)
        continuities = {}
        for s in c:
            continuities[s] = stroke_list.count(s)
        if c not in stroke_graph.edges:
            edge_geom = LineString(
                [
                    stroke_graph.nodes[c[0]]["geometry"],
                    stroke_graph.nodes[c[1]]["geometry"]
                ]
            )
            stroke_graph.add_edge(c[0], c[1], continuities=continuities, geometry=edge_geom)
        else:
            for s in c:
                stroke_graph.edges[c]["continuities"][s] += continuities[s]
            print(c, "already in graph!")
    print("\n")
# we want to add edges for all stroke IDs that co-occur on edges that share the same node in the primal graph
# [0, 1, 1] means: stroke0 has an endpoint here; stroke1 has a throughpoint here; we add the edge [0,1] in the strokes_graph, with the attribute 
# stroke = {0: "end", 1: "through"}

 

(1603585.6402153103, 6464428.773867372)
[0, 1, 1] {0, 1}
(0, 1)


(1603413.2063240695, 6464228.730248732)
[0, 1, 0, 2, 3] {0, 1, 2, 3}
(0, 1)
(0, 1) already in graph!
(0, 2)
(0, 3)
(1, 2)
(1, 3)
(2, 3)


(1603268.502117987, 6464060.781328565)
[4, 4, 5] {4, 5}
(4, 5)


(1603363.557831175, 6464031.88480676)
[4, 0, 4, 0] {0, 4}
(0, 4)


(1603607.3029882177, 6464181.852772597)
[1, 6, 1, 6] {1, 6}
(1, 6)


(1603226.9576840235, 6464160.158361825)
[2, 2, 4, 4] {2, 4}
(2, 4)


(1603039.9632033885, 6464087.491175889)
[2, 2, 7, 7] {2, 7}
(2, 7)


(1602887.2996537155, 6464029.975730775)
[2] {2}


(1602970.3773896934, 6464268.058242684)
[7] {7}


(1603090.513384159, 6463971.106984773)
[7, 8, 7] {8, 7}
(8, 7)


(1603317.7832565615, 6463836.796863219)
[7, 0, 7, 0] {0, 7}
(0, 7)


(1603202.3783404578, 6463872.287568242)
[7, 9, 7] {9, 7}
(9, 7)


(1603071.956425043, 6463729.978565)
[9] {9}


(1603650.450422848, 6464368.600601688)
[1, 6, 1, 6] {1, 6}
(1, 6)
(1, 6) already in graph!


(1603513.649900612

In [29]:
stroke_graph.edges(data=True)

EdgeDataView([(0, 1, {'continuities': {0: 3, 1: 3}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603707.107 6464238.854)>}), (0, 2, {'continuities': {0: 2, 2: 1}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603149.929 6464130.225)>}), (0, 3, {'continuities': {0: 2, 3: 1}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603342.343 6464406.368)>}), (0, 4, {'continuities': {0: 2, 4: 2}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603237.049 6464133.622)>}), (0, 7, {'continuities': {0: 2, 7: 2}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603264.658 6463848.976)>}), (1, 2, {'continuities': {1: 1, 2: 1}, 'geometry': <LINESTRING (1603707.107 6464238.854, 1603149.929 6464130.225)>}), (1, 3, {'continuities': {1: 1, 3: 1}, 'geometry': <LINESTRING (1603707.107 6464238.854, 1603342.343 6464406.368)>}), (1, 6, {'continuities': {1: 4, 6: 4}, 'geometry': <LINESTRING (1603707.107 6464238.854, 1603592.235 6464121.336)>}), (2, 3, {'continuities': {2: 1, 3: 1}, 'geometry': 

In [30]:
# get gdfs of points and lines
points_strokes, lines_strokes = momepy.nx_to_gdf(stroke_graph, points=True, lines=True)


In [31]:
lines_strokes["random_id"] = lines_strokes.index

In [32]:
len(stroke_gdf)

10

In [33]:
print(stroke_graph.edges(data=True))
m = stroke_gdf.explore(tiles="cartodb.positron", column = "stroke_id", name = "strokes", cmap = "tab20c", style_kwds={"weight":8})
lines_strokes.explore(m=m, 
    #column = "random_id", 
    name = "stroke_lines", 
    #cmap = "Blues", 
    style_kwds={"weight":8})
points_strokes.explore(m=m, marker_kwds={"radius":20}, name ="stroke nodes", color = "orange", opacity=0.2)
folium.LayerControl().add_to(m)
m

[(0, 1, {'continuities': {0: 3, 1: 3}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603707.107 6464238.854)>}), (0, 2, {'continuities': {0: 2, 2: 1}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603149.929 6464130.225)>}), (0, 3, {'continuities': {0: 2, 3: 1}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603342.343 6464406.368)>}), (0, 4, {'continuities': {0: 2, 4: 2}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603237.049 6464133.622)>}), (0, 7, {'continuities': {0: 2, 7: 2}, 'geometry': <LINESTRING (1603374.663 6464077.898, 1603264.658 6463848.976)>}), (1, 2, {'continuities': {1: 1, 2: 1}, 'geometry': <LINESTRING (1603707.107 6464238.854, 1603149.929 6464130.225)>}), (1, 3, {'continuities': {1: 1, 3: 1}, 'geometry': <LINESTRING (1603707.107 6464238.854, 1603342.343 6464406.368)>}), (1, 6, {'continuities': {1: 4, 6: 4}, 'geometry': <LINESTRING (1603707.107 6464238.854, 1603592.235 6464121.336)>}), (2, 3, {'continuities': {2: 1, 3: 1}, 'geometry': <LINESTRING (